### Not predicting correctly

Not a correct way as includes the laptimes, want to predict pre race

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))
from DB.models import init_db, Circuit, Season, RacingWeekend, Driver, Session, SessionResult, Lap


## Collect data into df

In [163]:
def create_dataframe():
    # initialize db connection and session
    db_engine, db_session = init_db()

    # query data from the database
    query = db_session.query(
        RacingWeekend.year,
        RacingWeekend.round,
        Circuit.circuit_name,
        Driver.driver_name,
        Driver.driver_short,
        Lap.lap_num,
        Lap.lap_time,
        Lap.tyre,
        Lap.pit,
        Session.session_type
    ).join(RacingWeekend.circuit) \
     .join(RacingWeekend.sessions) \
     .join(Session.laps) \
     .join(Lap.driver) \
     .join(RacingWeekend.season) \
     .all()

    # convert result to list of dicts
    data = []
    for row in query:
        data.append({
            'year': row.year,
            'round': row.round,
            'circuit_name': row.circuit_name,
            'driver_name': row.driver_name,
            'driver_short': row.driver_short,
            'lap_num': row.lap_num,
            'lap_time': row.lap_time,
            'tyre': row.tyre,
            'pit': row.pit,
            'session_type': row.session_type
        })

    # create dataframe
    df = pd.DataFrame(data)

    # encode 'tyre' and 'pit' as categories
    df['tyre'] = df['tyre'].astype('category')
    df['pit'] = df['pit'].astype('category')

    # one-hot encode categorical variables
    df = pd.get_dummies(df, columns=['circuit_name', 'driver_name', 'driver_short', 'tyre', 'session_type'], drop_first=True)

    return df

# create the dataframe and print it
df = create_dataframe()

print(df.columns.tolist())




['year', 'round', 'lap_num', 'lap_time', 'pit', 'circuit_name_Baku', 'circuit_name_Barcelona', 'circuit_name_Budapest', 'circuit_name_Hockenheim', 'circuit_name_Imola', 'circuit_name_Istanbul', 'circuit_name_Jeddah', 'circuit_name_Las Vegas', 'circuit_name_Le Castellet', 'circuit_name_Lusail', 'circuit_name_Marina Bay', 'circuit_name_Melbourne', 'circuit_name_Mexico City', 'circuit_name_Miami', 'circuit_name_Monaco', 'circuit_name_Monte Carlo', 'circuit_name_Montréal', 'circuit_name_Monza', 'circuit_name_Mugello', 'circuit_name_Nürburgring', 'circuit_name_Portimão', 'circuit_name_Sakhir', 'circuit_name_Shanghai', 'circuit_name_Silverstone', 'circuit_name_Singapore', 'circuit_name_Sochi', 'circuit_name_Spa-Francorchamps', 'circuit_name_Spielberg', 'circuit_name_Suzuka', 'circuit_name_São Paulo', 'circuit_name_Yas Island', 'circuit_name_Zandvoort', 'driver_name_Alexander Albon', 'driver_name_Andrea Kimi Antonelli', 'driver_name_Antonio Giovinazzi', 'driver_name_Arthur Leclerc', 'driver_n

In [164]:
# split data into training (2019, 2020, 2021) and testing (2022)
train_data = df[df['year'].isin([2019, 2020, 2021])]
test_data = df[df['year'] == 2022]

# define features (X) and target (y) for training
X_train = train_data.drop(columns=['pit'])
y_train = train_data['pit']

# define features (X) and target (y) for testing
X_test = test_data.drop(columns=['pit'])
y_test = test_data['pit']

# train the random forest model
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# predict on the 2022 test data
y_pred_test = rf_classifier.predict(X_test)

# add predictions to the test data
test_data['predicted_pit'] = y_pred_test

/tmp/ipykernel_11315/4025280255.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted_pit'] = y_pred_test


In [165]:
# evaluate the model

# accuracy
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy: {accuracy:.2f}")

# confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)
print("\nConfusion Matrix:")
print(conf_matrix)

# classification report
class_report = classification_report(y_test, y_pred_test)
print("\nClassification Report:")
print(class_report)

Accuracy: 0.95

Confusion Matrix:
[[46657   822]
 [ 1889   614]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.98      0.97     47479
         1.0       0.43      0.25      0.31      2503

    accuracy                           0.95     49982
   macro avg       0.69      0.61      0.64     49982
weighted avg       0.93      0.95      0.94     49982



In [ ]:
# specify driver and race to analyze
driver = 'RUS'
race_year = 2022
race_round = 6

# filter test data for the specific race and driver
df_race_driver = test_data[
    (test_data['session_type_Race'] == 1) &
    (test_data[f'driver_short_{driver}'] == 1) &
    (test_data['year'] == race_year) &
    (test_data['round'] == race_round)
]

# get laps where driver actually pitted
actual_pit_laps = df_race_driver[df_race_driver['pit'] == 1][['lap_num', 'pit']]

# get laps where model predicted a pit stop
predicted_pit_laps = df_race_driver[df_race_driver['predicted_pit'] == 1][['lap_num', 'predicted_pit']]


# print results
print(f"Driver {driver} - Actual Pit Stops in Race {race_year} Round {race_round}:")
print(actual_pit_laps)

print(f"\nDriver {driver} - Predicted Pit Stops in Race {race_year} Round {race_round}:")
print(predicted_pit_laps)

Driver RUS - Actual Pit Stops in Race 2022 Round 6:
Empty DataFrame
Columns: [lap_num, pit]
Index: []

Driver RUS - Predicted Pit Stops in Race 2022 Round 6:
        lap_num  predicted_pit
154856       14            1.0
154879       37            1.0
154894       52            1.0


: 